In [ ]:
import requests

from telegram.ext import Updater
import logging
from telegram.ext import CommandHandler
import math
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

def get_crypto_price(name):
    output = cg.get_price(ids=name, vs_currencies='usd')
    return float(output[name]['usd'])

def get_btc_price():
    response = requests.get('https://api.coindesk.com/v1/bpi/currentprice.json')
    data = response.json()
    # print(data)
    return float(data["bpi"]["USD"]["rate"].replace(',',''))

def get_index(pre_value,now_value,coin_price):

    new_fans = round(now_value - pre_value,4)
    new_fans_avg = round(new_fans/7,4)

    fans_daily_incre_ratio = round(new_fans_avg/pre_value,4)
    fans_weekly_incre_ratio = round(math.pow(1+fans_daily_incre_ratio,365)-1,4)

    fans_btcprice_ratio = round(now_value/coin_price,4)

    return pre_value,now_value,new_fans,new_fans_avg,fans_daily_incre_ratio,fans_weekly_incre_ratio,fans_btcprice_ratio,coin_price

def get_btc_index():
    url = "https://subredditstats.com/api/subreddit?name=bitcoin"
    json_str = requests.get(url).json()
    sub_count = json_str['subscriberCountTimeSeries']

    pre_value = sub_count[-8]['count']
    now_value = sub_count[-1]['count']
    coinprice = get_crypto_price('bitcoin')

    return get_index(pre_value,now_value,coinprice)

def get_crypto_index(url, coinprice):
    # url = "https://subredditstats.com/api/subreddit?name=bitcoin"
    json_str = requests.get(url).json()
    sub_count = json_str['subscriberCountTimeSeries']

    pre_value = sub_count[-8]['count']
    now_value = sub_count[-1]['count']
    # coinprice = get_btc_price()

    return get_index(pre_value,now_value,coinprice)

def btc_indx(update, context):
    output_text = ""
    pre_value,now_value,new_fans,new_fans_avg,fans_daily_incre_ratio,fans_weekly_incre_ratio,fans_btcprice_ratio,coin_price = get_btc_index()
    # print(pre_value,now_value,new_fans,new_fans_avg,fans_daily_incre_ratio,fans_weekly_incre_ratio,fans_btcprice_ratio)
    output_text = "<b>BTC价格</b> "+ str(coin_price) + "\n" + "<b>粉丝币价比</b> "+ str(fans_btcprice_ratio) + "\n" + "<b>7天粉丝复利年化增速</b> "+str(round(fans_weekly_incre_ratio*100,4))+"% \n" + "<b>粉丝日均增长率</b> "+str(round(fans_daily_incre_ratio*100,4))+"% \n"

    context.bot.send_message(chat_id=update.effective_chat.id, text=output_text, parse_mode = "HTML")

def eth_index(update, context):
    output_text = ""

    eth_url = "https://subredditstats.com/api/subreddit?name=Ethereum"
    eth_price = get_crypto_price('ethereum')
    pre_value,now_value,new_fans,new_fans_avg,fans_daily_incre_ratio,fans_weekly_incre_ratio,fans_btcprice_ratio,coin_price = get_crypto_index(eth_url, eth_price)

    # print(pre_value,now_value,new_fans,new_fans_avg,fans_daily_incre_ratio,fans_weekly_incre_ratio,fans_btcprice_ratio)
    output_text = "<b>ETH价格</b> "+ str(coin_price) + "\n" + "<b>粉丝币价比</b> "+ str(fans_btcprice_ratio) + "\n" + "<b>7天粉丝复利年化增速</b> "+str(round(fans_weekly_incre_ratio*100,4))+"% \n" + "<b>粉丝日均增长率</b> "+str(round(fans_daily_incre_ratio*100,4))+"% \n"

    context.bot.send_message(chat_id=update.effective_chat.id, text=output_text, parse_mode = "HTML")

def bnb_index(update, context):
    output_text = ""
    bnb_url = "https://subredditstats.com/api/subreddit?name=binance"
    bnb_price = get_crypto_price('binancecoin')
    pre_value,now_value,new_fans,new_fans_avg,fans_daily_incre_ratio,fans_weekly_incre_ratio,fans_btcprice_ratio,coin_price = get_crypto_index(bnb_url, bnb_price)
    # print(pre_value,now_value,new_fans,new_fans_avg,fans_daily_incre_ratio,fans_weekly_incre_ratio,fans_btcprice_ratio)
    output_text = "<b>BNB价格</b> "+ str(coin_price) + "\n" + "<b>粉丝币价比</b> "+ str(fans_btcprice_ratio) + "\n" + "<b>7天粉丝复利年化增速</b> "+str(round(fans_weekly_incre_ratio*100,4))+"% \n" + "<b>粉丝日均增长率</b> "+str(round(fans_daily_incre_ratio*100,4))+"% \n"

    context.bot.send_message(chat_id=update.effective_chat.id, text=output_text, parse_mode = "HTML")

def doge_index(update, context):
    output_text = ""
    bnb_url = "https://subredditstats.com/api/subreddit?name=dogecoin"
    bnb_price = get_crypto_price('dogecoin')
    pre_value,now_value,new_fans,new_fans_avg,fans_daily_incre_ratio,fans_weekly_incre_ratio,fans_btcprice_ratio,coin_price = get_crypto_index(bnb_url, bnb_price)
    # print(pre_value,now_value,new_fans,new_fans_avg,fans_daily_incre_ratio,fans_weekly_incre_ratio,fans_btcprice_ratio)
    output_text = "<b>DOGE价格</b> "+ str(coin_price) + "\n" + "<b>粉丝币价比</b> "+ str(fans_btcprice_ratio) + "\n" + "<b>7天粉丝复利年化增速</b> "+str(round(fans_weekly_incre_ratio*100,4))+"% \n" + "<b>粉丝日均增长率</b> "+str(round(fans_daily_incre_ratio*100,4))+"% \n"

    context.bot.send_message(chat_id=update.effective_chat.id, text=output_text, parse_mode = "HTML")

def btc_pic_index(update, context):
    context.bot.send

def main():
    logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                     level=logging.INFO)
    #需要使用bot 的 token
    token_str = ""
    updater = Updater(token= token_str, use_context=True)
    dispatcher = updater.dispatcher
    start_handler = CommandHandler('btc', btc_indx)
    start_handler2 = CommandHandler('eth', eth_index)
    start_handler3 = CommandHandler('bnb', bnb_index)
    start_handler4 = CommandHandler('doge', doge_index)


    dispatcher.add_handler(start_handler)
    dispatcher.add_handler(start_handler2)
    dispatcher.add_handler(start_handler3)
    dispatcher.add_handler(start_handler4)
    updater.start_pol0ling()
main()
# print(get_crypto_price('bitcoin'))
# print(get_crypto_price('dogecoin'))
# # print(cg.get_coins_list())
# print(get_crypto_price('ethereum'))
# print(get_crypto_price('binancecoin'))
# print(get_crypto_price('pancakeswap-token'))